In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from PIL import Image
from torchmetrics.classification import BinaryAUROC
import pandas as pd
import os
import clip
import torch.nn as nn
import torch.nn.functional as F

import IPython.display
import matplotlib.pyplot as plt
from PIL import Image

from collections import OrderedDict

print("Torch version:", torch.__version__)


Torch version: 1.13.0+cu117


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
HOME = os.environ.get("HOME")
images_path = f"{HOME}/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/"
annotations_path = f"{HOME}/.cache/torch/mmf/data/datasets/hateful_memes/defaults/annotations/"

In [4]:
class HMDataset(Dataset):
    def __init__(self, images_path: str, annotation_path: str, image_transform=None, text_transform=None) -> None:
        self.images_path = images_path
        self.annotation_path = annotation_path
        self.image_transform = image_transform
        self.text_transform = text_transform
        assert self.annotation_path.endswith(".jsonl"), f"Invalid annotation file format. Format should be '.jsonl', not {self.annotation_path.split('.')[0]}"
        self.annotation: pd.DataFrame = pd.read_json(self.annotation_path, lines=True)      

    def __len__(self):
        return self.annotation.shape[0]

    def __getitem__(self, index):
        img_path = os.path.join(self.images_path.split('img')[0], self.annotation.loc[index,"img"])
        image = Image.open(img_path).convert("RGB")
        text = self.annotation.loc[index,"text"]
        label = self.annotation.loc[index,"label"]
        if self.image_transform:
            image = self.image_transform(image)
        if self.text_transform:
            text = self.text_transform(text)
        return image, text, torch.tensor(label)

In [5]:
clip.available_models()


['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [6]:
model, preprocess = clip.load("ViT-L/14@336px")
model.to(device).eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

100%|███████████████████████████████████████| 891M/891M [02:54<00:00, 5.35MiB/s]


Model parameters: 427,944,193
Input resolution: 336
Context length: 77
Vocab size: 49408


In [7]:
class HMMLP(nn.Module):

    def __init__(self, n_in=768*2, n_out=1, ) -> None:
        super().__init__()

        self.fc1 = nn.Linear(n_in, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, n_out)

    def forward(self, x):
        
        # x = self.fc1(x)
        x = F.gelu(self.fc1(x))
        x = F.gelu(self.fc2(x))
        x = F.gelu(self.fc3(x))
        x = self.fc4(x)

        return x

In [8]:
batch_size = 32

def text_preprocess(text):
    return clip.tokenize(text, truncate=True)

hm_train_dataset = HMDataset(images_path, f"{annotations_path}/train_v2.jsonl", image_transform=preprocess, text_transform=text_preprocess)
hm_test_dataset = HMDataset(images_path, f"{annotations_path}/test_unseen.jsonl", image_transform=preprocess, text_transform=text_preprocess)
hm_val_dataset = HMDataset(images_path, f"{annotations_path}/dev_unseen.jsonl", image_transform=preprocess, text_transform=text_preprocess)

train_dataloader = DataLoader(hm_train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(hm_test_dataset, batch_size=batch_size, shuffle=False)
val_dataloader = DataLoader(hm_val_dataset, batch_size=batch_size, shuffle=False)


In [9]:
from tqdm import tqdm
net = HMMLP()

model_path = 'base_model.pt'
net = net.to(device)
criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.AdamW(net.parameters(), lr=0.01)

epochs = 20
print_every = 50

for epoch in range(epochs):
    
    running_loss = 0
    for i, data in enumerate(tqdm(train_dataloader), 0):
        images, texts, labels = data
        images = images.to(device)
        texts = texts.to(device)
        labels = labels.float().squeeze().to(device)

        with torch.no_grad():
            images = model.encode_image(images) # input_dim: batch_size x 3 x H x W; output_dim: batch_size x 512
            texts = model.encode_text(texts.squeeze()) # input_dim: batch_size x 77; output_dim: batch_size x 512
        
        fused_images_texts = torch.hstack((images,texts))
        fused_images_texts.requires_grad_()
        fused_images_texts = fused_images_texts.float()

        optimizer.zero_grad()

        # Forward pass on the fused data

        output = net(fused_images_texts)

        loss = criterion(output.squeeze(), labels)

        # Compute gradient
        loss.backward()
        # Update weight
        optimizer.step()

        running_loss += loss.item()

        if i%print_every==(print_every - 1): 
            
            print(f"[Epoch {epoch + 1}, step {i+1:3d}] loss: {running_loss/print_every:.5f}")
            running_loss = 0.0
    ## Switch to eval mode
    torch.save(net.state_dict(), model_path)
    net.eval()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0

    preds_all_val = torch.tensor([]).cuda()
    labels_all_val = torch.tensor([]).cuda()
    for i, data in enumerate(val_dataloader, 0):
        
        images, texts, labels = data
        images = images.to(device)
        texts = texts.to(device)
        labels = labels.float().squeeze().to(device)

        with torch.no_grad():
            images = model.encode_image(images) # input_dim: batch_size x 3 x H x W; output_dim: batch_size x 512
            texts = model.encode_text(texts.squeeze()) # input_dim: batch_size x 77; output_dim: batch_size x 512
        
        fused_images_texts = torch.hstack((images,texts))
        fused_images_texts.requires_grad_()
        fused_images_texts = fused_images_texts.float()

        with torch.no_grad():
            output = net(fused_images_texts)

        loss = criterion(output.squeeze(), labels)

        running_loss += loss.item()

        correct_preds += sum(torch.sigmoid(output).squeeze().round()==labels)
        total_preds += len(labels)

        preds_all_val = torch.cat((preds_all_val, torch.sigmoid(output).squeeze()))
        labels_all_val = torch.cat((labels_all_val, labels))

    auroc = BinaryAUROC()
    auroc_score = auroc(preds_all_val, labels_all_val.int())
    print(f"\n[Epoch {epoch +1}, step {i+1:3d}] val loss: {running_loss/i+1:.5f} "
    f"accuracy: {torch.mean(preds_all_val.round()==labels_all_val, dtype=torch.float32)} auroc: {auroc_score}\n")
    # f"accuracy: {correct_preds/total_preds} auroc: {auroc_score}\n")

print("Finished Training!")

 18%|███████▌                                  | 50/279 [01:07<04:48,  1.26s/it]

[Epoch 1, step  50] loss: 0.56839


 36%|██████████████▋                          | 100/279 [02:10<03:49,  1.28s/it]

[Epoch 1, step 100] loss: 0.52300


 54%|██████████████████████                   | 150/279 [03:13<02:43,  1.26s/it]

...

[Epoch 20, step 200] loss: 0.03239


 90%|████████████████████████████████████▋    | 250/279 [04:14<00:28,  1.02it/s]

[Epoch 20, step 250] loss: 0.05485


100%|█████████████████████████████████████████| 279/279 [04:44<00:00,  1.02s/it]



[Epoch 20, step  17] val loss: 2.42863 accuracy: 0.6851851940155029 auroc: 0.7346985340118408

Finished Training!


In [10]:
correct_preds = 0
total_preds = 0

preds_all_val = torch.tensor([]).cuda()
labels_all_val = torch.tensor([]).cuda()
for i, data in enumerate(test_dataloader, 0):
    images, texts, labels = data
    images = images.to(device)
    texts = texts.to(device)
    labels = labels.float().squeeze().to(device)

    with torch.no_grad():
        images = model.encode_image(images) # input_dim: batch_size x 3 x H x W; output_dim: batch_size x 512
        texts = model.encode_text(texts.squeeze()) # input_dim: batch_size x 77; output_dim: batch_size x 512
    
    fused_images_texts = torch.hstack((images,texts))
    fused_images_texts.requires_grad_()
    fused_images_texts = fused_images_texts.float()

    with torch.no_grad():
        output = net(fused_images_texts)

    loss = criterion(output.squeeze(), labels)

    running_loss += loss.item()

    correct_preds += sum(torch.sigmoid(output).squeeze().round()==labels)
    total_preds += len(labels)

    preds_all_val = torch.cat((preds_all_val, torch.sigmoid(output).squeeze()))
    labels_all_val = torch.cat((labels_all_val, labels))

auroc_score = auroc(preds_all_val, labels_all_val.int())
print(f"accuracy: {torch.mean(preds_all_val.round()==labels_all_val, dtype=torch.float32)} auroc: {auroc_score}\n")



accuracy: 0.733500063419342 auroc: 0.79988694190979

